In [1]:
!pip install NLTK
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
df = pd.read_csv('main.csv')

df

,title,label,label_score
0,"Masuk Radar Pilwalkot Medan, Menantu Jokowi Be...",non-clickbait,0
1,Malaysia Sudutkan RI: Isu Kabut Asap hingga In...,non-clickbait,0
2,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1
3,"Kemensos Salurkan Rp 7,3 M bagi Korban Kerusuh...",non-clickbait,0
4,"Terkait Mayat Bayi Mengenaskan di Tangerang, S...",non-clickbait,0
...,...,...,...
14995,"Tolak RUU Pertanahan, Ribuan Petani Siap Gelar...",non-clickbait,0
14996,Ada Niat Tambah Momongan Tanpa Ikut Program Ha...,clickbait,1
14997,"Beredar Isu Internet Papua Diblokir Lagi, Telk...",non-clickbait,0
14998,"TXT Akan Segera Comeback, Soobin Akui Gatal I...",clickbait,1


In [4]:
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    # remove non-alphanumeric characters
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    # tokenize text into words
    words = word_tokenize(text)
    # remove stopwords
    words = [word for word in words if word not in stopwords.words('indonesian')]
    # stem words using Sastrawi stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = [stemmer.stem(word) for word in words]
    # join words back into text
    text = ' '.join(words)
    return text

In [5]:
# preprocess text in dataset
df['title'] = df['title'].apply(preprocess_text)

In [6]:
df

,title,label,label_score
0,masuk radar pilwalkot medan menantu jokowi tem...,non-clickbait,0
1,malaysia sudut ri isu kabut asap invasi babi,non-clickbait,0
2,viral driver ojol bekas pesan makan pakai sepeda,clickbait,1
3,kemensos salur rp m korban rusuh sosial papua,non-clickbait,0
4,kait mayat bayi enas tangerang pria tangkap po...,non-clickbait,0
...,...,...,...
14995,tolak ruu tanah ribu tani gelar aksi istanadpr...,non-clickbait,0
14996,niat momong program hamil fedi nuril pengin ya...,clickbait,1
14997,edar isu internet papua blokir telkom warga pe...,non-clickbait,0
14998,txt comeback soobin aku gatal pamer spoiler,clickbait,1


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [8]:
# Split dataset into train and test set
X_train, X_test, y_train, y_test = train_test_split(df['title'], df['label'], test_size=0.2, random_state=42)

In [9]:
X_train

9839             kota milik transportasi konsep smart city
9680             chapter one piece ungkap mati orang luffy
7093           ayu ting ting enggan raya ulang enam bilqis
11293                   komisi iv dpr tuju anggar tan rp t
820      heboh babi masuk masjid ibuibu teriak si babi ...
                               ...                        
5191                     fakta kejut ungkap celaka nganjuk
13418    sukanto tanoto konglomerat lahan hti kawasan c...
5390     aji santoso kenang bj habibie semangat timnas ...
860      polisi halau aksi uncen jayapura mahasiswa eks...
7270               ekonomi roket jokowi omong kosong bukti
Name: title, Length: 12000, dtype: object

In [10]:
X_test

11499         arkeolog temu situs david vs goliath alkitab
6475     tampil classy rok tutu inspirasi hijab ala agh...
13167    hasil china open marcuskevin sukses bayar luna...
862      pasang ungsi rohingya tewas baku tembak bangla...
5970         jadwal wakil indonesia semifinal vietnam open
                               ...                        
6216     berkat imej seksi pamela safitri laris manis d...
3585      bela rossi insiden kualifikasi motogp san marino
10023    palsu terang akte otentik henry j gunawan istr...
14044    bebby fey tegas sang bunda jatuh sakit atta ha...
12965            itinerary tokyo malam cocok traveler mula
Name: title, Length: 3000, dtype: object

In [11]:
# TF-IDF vectorization
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [13]:
# KNN classification with k-fold cross validation
k_range = list(range(1, 26))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_tfidf, y_train, cv=5, scoring='accuracy')
    k_scores.append(scores.mean())
optimal_k = k_range[k_scores.index(max(k_scores))]

knn = KNeighborsClassifier(n_neighbors=optimal_k)
knn.fit(X_train_tfidf, y_train)
accuracy = knn.score(X_test_tfidf, y_test)
print('Optimal k: {}'.format(optimal_k))
print('Accuracy: {:.2f}%'.format(accuracy*100))

Optimal k: 21
Accuracy: 72.53%


In [14]:
import pickle
# save model to file
with open('clickbait_knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

In [18]:
# fit TF-IDF vectorizer to data
tfidf.fit(df['title'])

# save TF-IDF vectorizer checkpoint
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf, f)